## Loan Recommender

In [9]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
import ast

In [10]:
# Initialize the LLM (using Ollama in this example).
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

In [11]:

# Create a prompt template for extracting parameters.
extraction_prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
        "Given the following user input:\n"
        "\"{user_input}\"\n\n"
        "Extract the following parameters as a JSON object:\n"
        "- deposit_amount: The amount of money deposited (if mentioned; otherwise, None).\n"
        "- deposit_duration: The duration of the deposit in months (should be between 1 and 12; if mentioned; otherwise, None).\n"
        "- loan_amount: The total loan amount (if mentioned; maximum allowed is 300000000; otherwise, None).\n"
        "- credit_score: One of the following values: A, B, C, D, E, None (if mentioned; otherwise, None).\n"
        "- number_of_installments: Allowed values are 4, 6, 9, 12, 18, 24, 36, 48 (if mentioned; otherwise, None).\n"
        "- interest_rate: Allowed values are 0, 2, 4, 18, 23 (if mentioned; otherwise, None).\n\n"
        "If a parameter is not mentioned in the input, set its value to None.\n\n"
        "Example output:\n"
        "{{\n"
        '  "deposit_amount": 5000000,\n'
        '  "deposit_duration": 6,\n'
        '  "loan_amount": "None",\n'
        '  "credit_score": "None",\n'
        '  "number_of_installments": "None",\n'
        '  "interest_rate": "None"\n'
        "}}"
    )
)
# Create the extraction chain.
extraction_chain = LLMChain(llm=llm, prompt=extraction_prompt, verbose=True)

#

In [12]:

# Define the Function to Parse the LLM Output

def parse_extraction_result(result_str: str) -> dict:
    """
    Extract the dictionary from the LLM output.
    First, find the substring between the first '{' and the last '}'.
    Then try to parse that substring as JSON.
    If that fails (for example, due to single quotes), fall back to ast.literal_eval.
    """
    start = result_str.find('{')
    end = result_str.rfind('}') + 1
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the output.")
    json_str = result_str[start:end]
    try:
        return json.loads(json_str)
    except Exception:
        try:
            return ast.literal_eval(json_str)
        except Exception as e2:
            raise ValueError(f"Error parsing extraction result: {e2}")


In [16]:


# Example user input.
# user_input = (
#     "I deposited 5 million Tomans for 6 months. Also, I took a loan of 250000000 from Bank X. "
#     "My credit score is A. I have 12 installments "
# )


user_input = (
    "من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم توی اقساط ۹ ماهه پرداختش کنم"
)

# user_input = (
#     "من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم تعداد ۸ قسط پرداختش کنم"
# )



# and the interest rate is 4.
# Run the extraction chain.
extraction_result = extraction_chain.run({"user_input": user_input})
print("Extraction result:")
print(extraction_result)

# Function to parse the LLM output into a Python dictionary.
def parse_output(result_str: str) -> dict:
    # Find the first '{' and the last '}' in the output.
    start = result_str.find('{')
    end = result_str.rfind('}') + 1
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the output.")
    json_str = result_str[start:end]
    try:
        return json.loads(json_str)
    except Exception:
        # Fallback to ast.literal_eval if JSON parsing fails.
        return ast.literal_eval(json_str)

# Parse the extracted output.
parsed_params = parse_output(extraction_result)
print("Parsed parameters:")
print(parsed_params)




> Entering new LLMChain chain...
Prompt after formatting:
Given the following user input:
"من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم توی اقساط ۹ ماهه پرداختش کنم"

Extract the following parameters as a JSON object:
- deposit_amount: The amount of money deposited (if mentioned; otherwise, None).
- deposit_duration: The duration of the deposit in months (should be between 1 and 12; if mentioned; otherwise, None).
- loan_amount: The total loan amount (if mentioned; maximum allowed is 300000000; otherwise, None).
- credit_score: One of the following values: A, B, C, D, E, None (if mentioned; otherwise, None).
- number_of_installments: Allowed values are 4, 6, 9, 12, 18, 24, 36, 48 (if mentioned; otherwise, None).
- interest_rate: Allowed values are 0, 2, 4, 18, 23 (if mentioned; otherwise, None).

If a parameter is not mentioned in the input, set its value to None.

Example output:
{
  "deposit_amount": 5000000,
  "deposit_duration": 6,
  "loan_amount": "None",
  "credit_score": "